In [25]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import torch
import glob
import math
import numpy as np
from medpy.io import load
import pyvista
import pymeshfix
import pyacvd
import pymesh
import matplotlib
import matplotlib.pyplot as plt
from dataset_vessel import build_vessel_data
from monai.data import DataLoader
from torch import nn
from skimage.morphology import skeletonize_3d
from skimage.measure import marching_cubes_lewiner
from scipy.sparse import csr_matrix
from models.detr_transformer_3D import build_detr_transformer
from models.swin_transformer_3D import build_swin_transformer
# from models.matcher import build_matcher
from losses import SetCriterion
from models import build_model
import pyvista
import networkx as nx
import dgl
import dgl.function as fn
# %matplotlib widget
import yaml
import sys
sys.path.append("..")
import json
from models.position_encoding import PositionalEncoding3D
from torch.utils.tensorboard import SummaryWriter
from scipy import ndimage
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from skimage.morphology import skeletonize_3d
from utils import save_input, save_output
from utils import Bresenham3D
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
class obj:
    def __init__(self, dict1):
        self.__dict__.update(dict1)
        
def dict2obj(dict1):
    return json.loads(json.dumps(dict1), object_hook=obj)

In [5]:
config_file = "/media/Drives/Data/transformer_graph_gen/configs/synth_3D_swin_base_patch4_window7_64_deform_detr.yaml"

with open(config_file) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

config = dict2obj(config)

In [6]:
data_dir = '/media/Drives/Media/vessel_centerline/3D_vessel_data/'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
device = "cuda"

In [8]:
dataset = build_vessel_data(config, device=device)

In [9]:
def image_graph_collate(batch):
    images = torch.cat([item_ for item in batch for item_ in item[0]], 0).contiguous()
    points = [item_ for item in batch for item_ in item[1]]
    edges = [item_ for item in batch for item_ in item[2]]
    return [images, points, edges]

In [10]:
mn_dataset_loader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=lambda x: tuple([y_.to(device) for y_ in x_] for x_ in image_graph_collate(x)))

In [11]:
for images, points, edges in mn_dataset_loader:
    images = torch.stack(images)
    print(len(points), images.shape, points[0].shape)
    break

6 torch.Size([6, 1, 64, 64, 64]) torch.Size([37, 3])


In [60]:
i = 3
patch = images[i,0,...].cpu().numpy()
coord = points[i].cpu().numpy()
edge = edges[i].cpu().numpy()

In [61]:
# keep only the largest graph (in length or voxel space?)
dilated_labels = patch #np.zeros(patch.shape)
# dilated_labels[5:-5, 5:-5, 5:-5] = ndimage.morphology.binary_dilation(patch[5:-5, 5:-5, 5:-5], structure=np.ones([3,3,3]))
# skel = skeletonize_3d(dilated_labels)
image_labeled, n_comp_image = ndimage.label(dilated_labels>0.0, structure=np.ones([3,3,3]))
# skel_labeled, n_comp_skel = ndimage.label(skel>0.0, structure=np.ones([3,3,3]))
# sizes = ndimage.sum(skel, skel_labeled, range(n_comp_skel + 1))
# label_largest_image = np.argmax(sizes)

# find largest graph segment in graph and in skeleton and see if they match
dist_adj = np.zeros((coord.shape[0], coord.shape[0]))
dist_adj[edge[:,0], edge[:,1]] = np.sum((coord[edge[:,0],:]-coord[edge[:,1],:])**2, 1)
dist_adj[edge[:,1], edge[:,0]] = np.sum((coord[edge[:,0],:]-coord[edge[:,1],:])**2, 1)

graph = csr_matrix(np.triu(dist_adj>0.0))
n_comp_graph, graph_labeled = connected_components(csgraph=graph, directed=False, return_labels=True)

if n_comp_graph==n_comp_image:
    print("All is good")

dist = np.zeros(n_comp_graph)
for i in range(n_comp_graph):
    idx = np.where(graph_labeled==i)[0]
    dist[i] = np.sum(np.triu(dist_adj[np.ix_(idx,idx)]))
    
max_subgraph_label = np.argmax(dist)
max_subgraph_coord = np.int32(coord[graph_labeled==max_subgraph_label,:]*(np.array(patch.shape)-1))
img_label_max_subgraph = image_labeled[max_subgraph_coord[:,0], max_subgraph_coord[:,1], max_subgraph_coord[:,2]]
new_patch = 1.0*(image_labeled==np.median(img_label_max_subgraph[img_label_max_subgraph>0.0]))

all_coord = np.int32(coord*(np.array(patch.shape)-1))
label = new_patch[all_coord[:,0], all_coord[:,1], all_coord[:,2]]

# keep the nodes which lie in the largest compnent and their subgraph
keep_graph_label = np.array(np.unique(graph_labeled[label>0]))
if keep_graph_label.ndim==0:
    final_graph = (graph_labeled==keep_graph_label)
else:
    final_graph = np.zeros(graph_labeled.shape, dtype=np.bool)
    for ind in keep_graph_label:
        final_graph += (graph_labeled==ind)
new_dist_adj = dist_adj[np.ix_(final_graph,final_graph)]
new_coord = coord[final_graph,:]
new_edge = np.array(np.where(np.triu(new_dist_adj)>0)).T
new_patch = patch*new_patch

All is good


In [62]:
path = ('pre')
save_input(path, 1, patch, coord, edge)
path = ('post1')
save_input(path, 1, new_patch, new_coord, new_edge)

(19, 3)
(10, 3)


In [72]:
def dotproduct(v1, v2):
    return sum((a*b) for a, b in zip(v1, v2))

def length(v):
    return math.sqrt(dotproduct(v, v))

def angle(v1, v2):
    return np.arccos(dotproduct(v1, v2) / (length(v1) * length(v2)))

In [64]:
start = True
node_mask = np.ones(new_coord.shape[0], dtype=np.bool)
while start:
    degree = (new_dist_adj>0).sum(1)
    deg_2 = list(np.where(degree==2)[0])
    for n, idx in enumerate(deg_2):
        deg_2_neighbor = np.where(new_dist_adj[idx,:]>0)[0]

        p1 = new_coord[idx,:]
        p2 = new_coord[deg_2_neighbor[0],:]
        p3 = new_coord[deg_2_neighbor[1],:]
        l1 = p2-p1
        l2 = p3-p1

        node_angle = angle(l1,l2)*180 / math.pi

        if node_angle>160:
#             connecting_line = np.array(Bresenham3D(np.int32(p2*np.array(new_patch.shape)), np.int32(p3*np.array(new_patch.shape))))
#             if (new_patch[connecting_line[:,0], connecting_line[:,1], connecting_line[:,2]]>0).all():
            node_mask[idx]=False
            new_dist_adj[deg_2_neighbor[0], deg_2_neighbor[1]] = np.sum((p2-p3)**2)
            new_dist_adj[deg_2_neighbor[1], deg_2_neighbor[0]] = np.sum((p2-p3)**2)

            new_dist_adj[idx, deg_2_neighbor[0]] = 0.0
            new_dist_adj[deg_2_neighbor[0], idx] = 0.0
            new_dist_adj[idx, deg_2_neighbor[1]] = 0.0
            new_dist_adj[deg_2_neighbor[1], idx] = 0.0
            break
        elif n==len(deg_2)-1:
            start = False

new_coord = new_coord[node_mask,:]
new_dist_adj = new_dist_adj[np.ix_(node_mask, node_mask)]
new_edge = np.array(np.where(np.triu(new_dist_adj)>0)).T

all: 1 [ 2 10] 141.6501785806879
all: 3 [0 7] 135.73562885208713
all: 4 [2 8] 172.96645316964396
corrected: 4 [2 8] 172.96645316964396
all: 1 [ 2 10] 141.6501785806879
all: 3 [0 7] 135.73562885208713
all: 5 [0 2] 159.64293565946517
corrected: 5 [0 2] 159.64293565946517
all: 1 [ 2 10] 141.6501785806879
all: 3 [0 7] 135.73562885208713
all: 6 [0 9] 151.4893583886943
corrected: 6 [0 9] 151.4893583886943
all: 1 [ 2 10] 141.6501785806879
all: 3 [0 7] 135.73562885208713


In [65]:
path = ('post2')
save_output(path, 1, new_coord, new_edge)

(7, 3)


In [73]:
p1 = [ 0.01931787, -0.0222708,  0.02970314] 
p2 = [-0.01931787,  0.02227092, -0.02970266]

In [74]:
angle(p1,p2)*180 / math.pi

179.99943856089104

In [176]:
# straighten the graph by removing redundant nodes
degree = full_adj.sum(1)
deg_2 = np.where(degree==2)
deg_2_neighbor = np.where(full_adj[deg_2,:])
p1 = coord[deg_2,:]
p2 = coord[deg_2_neighbor[0],:]
p3 = coord[deg_2_neighbor[1],:]

l1 = p1-p3
l2 = p2-p3

node_angle = angle(l1,l2)*180 / math.pi

remove_node = np.where(node_angle>170)
while remove_node.sum()>0:
    full_adj[deg_2[remove_node], deg_2[deg_2_neighbor[0][remove_node]]] = 0.0
    full_adj[deg_2[deg_2_neighbor[0][remove_node]], deg_2[remove_node]] = 0.0
    full_adj[deg_2[remove_node], deg_2[deg_2_neighbor[1][remove_node]]] = 0.0
    full_adj[deg_2[deg_2_neighbor[1][remove_node]], deg_2[remove_node]] = 0.0
    full_adj[deg_2[deg_2_neighbor[0][remove_node]], deg_2[deg_2_neighbor[1][remove_node]]] = 1.0
    full_adj[deg_2[deg_2_neighbor[1][remove_node]], deg_2[deg_2_neighbor[0][remove_node]]] = 1.0

    degree = full_adj.sum(1)
    deg_2 = np.where(degree==2)
    deg_2_neighbor = np.where(full_adj[deg_2,:])
    p1 = coord[deg_2,:]
    p2 = coord[deg_2_neighbor[0],:]
    p3 = coord[deg_2_neighbor[1],:]

    l1 = p1-p3
    l2 = p2-p3

    node_angle = angle(l1,l2)*180 / math.pi

    remove_node = np.where(node_angle>170)

new_nodes = np.where(degree>0)
new_coord = coord[new_nodes,:]
new_adj = full_adj[new_nodes, new_nodes]
new_edge = np.where(np.triu(new_adj)>0)


mod_patch_coord_list.append(new_coord)
mod_patch_edge_list.append(new_edge)

ValueError: operands could not be broadcast together with shapes (1,9,3) (18,3) 

(25, 3)


NameError: name 'new_edge' is not defined

In [24]:
# for i,data in enumerate(dataset):
# #     save_patch(data[0][1], data[1][1], data[2][1])
#     print(i, data[0][1].shape, data[1][1].shape, data[2][1].shape)
# #     save_patch(data[0][1], data[1][1], data[2][1])
#     break

In [29]:
position_embedding = PositionalEncoding3D(channels=config.MODEL.DECODER.HIDDEN_DIM)
query_embed = nn.Embedding(num_embeddings=100, embedding_dim=config.MODEL.DECODER.HIDDEN_DIM).weight.to(device)

In [37]:
enocder = build_swin_transformer(config).to(device)

In [38]:
feat_list, mask_list, pos_list = enocder(images.to(device), position_embedding, return_interm_layers=False)

In [44]:
decoder = build_detr_transformer(config).to(device)

In [45]:
out = decoder(feat_list[0].flatten(2).flatten(2).transpose(1, 2), None, query_embed, pos_list[0].transpose(1, 2))

torch.Size([100, 2, 1024])
torch.Size([100, 2, 1024])
torch.Size([100, 2, 1024])
torch.Size([100, 2, 1024])
torch.Size([100, 2, 1024])
torch.Size([100, 2, 1024])


In [47]:
out.shape

torch.Size([2, 100, 1024])

In [ ]:
h = decoder(out, tgt_graph, tgt_node, tgt_edges)

In [88]:
indices = matcher(h, points)